## 12.5 Wstrzykiwanie wartości (Value Injection)

Zajmijmy się teraz przekazaniem wartości przez konstruktor w czasie wykonania. Naszym przykładem dalej będzie `Computer`, tym razem będzie posiadał tylko jeden komponent - `GPU`. Klasa `GPU` będzie przyjmować jeden paramtetr przez konstruktor - liczbę rdzeni typu `Int`.

In [ ]:
public class Computer {

    private GPU gpu;

    @Inject
    public Computer(GPU gpu) {
        this.gpu = gpu;
    }

    public String getGpuCores(){
        return String.valueOf(gpu.getGpuCores());
    }
}


public class GPU {

    private int gpuCores;

    public GPU(int gpuCores) {
        this.gpuCores = gpuCores;
    }

    public int getGpuCores() {
        return gpuCores;
    }
}

Teraz nie możemy wykorzystać adnotacji `@Inject` przy konstruktorze klasy `GPU` - liczba rdzeni będzie znana w czasie wykonania aplikacji (może być podana przez użytkownika). Musimy manualnie wywołać konstruktor i w jakiś sposób przekazać wartość `gpuCores`.

Tym razem nasza klasa z adnotacją `@Module` będzie posiadała konstruktor w którym przekażemy wartość `gpuCores`

In [ ]:
@Module
public class GpuModule {
    
    private int gpuCores;

    public GpuModule(int gpuCores) {
        this.gpuCores = gpuCores;
    }
    
    @Provides
    GPU provideGpu(){
        return new GPU(gpuCores);
    }
}

Naszx `ComputerComponent` nie zawiera żadnych nowych elementów.

In [ ]:
@Component(modules = GpuModule.class)
public interface ComputerComponent {
    void inject(MainActivity mainActivity);
}

Zmiany natomiast pojawią się w automatycznie wygenerowanej klasie `DaggerComputerComponent`. Nie mamy dostępnej metody `create`, teraz musimy manualnie wywołać konstruktor klasy `GpuModule`. Do tego automatycznie generowana jest implementacja wzorca projektowego [budowniczy](https://refactoring.guru/pl/design-patterns/builder).

In [ ]:
ComputerComponent component = DaggerComputerComponent.builder()
    .gpuModule(new GpuModule(8))  // możemy przekazać liczbę rdzeni w czasie wykonania
    .build();

Tak jak w poprzednich przykładach, trworzymy instancję obiektu o typie interfejsu `ComputerComponent`, następnie wywołujemy metodę `inject`

In [ ]:
public class MainActivity extends AppCompatActivity {

    @Inject
    public Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        ComputerComponent component = DaggerComputerComponent.builder()
                .gpuModule(new GpuModule(8))
                .build();

        component.inject(this);
        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.getGpuCores());
    }
}

### Metoda dostarczająca

W klasie `GpuModule` przekazujemy jawnie wartość `gpuCores`, możemy jednak zaimplementować metodę dostarczającą dla tej zmiennej - dzięki temu możemy tą jedną wartość dostarczyć do wielu obiektów - jednym z zastosowań w androidzie jest przekazanie **applicationContext**.

Powróćmy do klasy `GpuModule` i dodajmy metodę `provideGpuCores`

In [ ]:
@Provides
int provideGpuCores(){
    return gpuCores;
}

Teraz możemy wstrzyknąć za pomocą konstruktora `GPU`

In [ ]:
public class GPU {
    private int gpuCores;
    
    @Inject
    public GPU(int gpuCores){
        thi.gpuCores = gpuCores;
    }
}

Teraz z kolei, możemy zmodyfikować metodę `provideGpu` klasy `GpuModule` i przekazać `gpu` przez parametr.

In [ ]:
@Provides
GPU provideGpu(GPU gpu){
    return gpu
}

Tutaj konieczne jest  pamiętanie i jednej istotnej rzeczy - `Dagger2` nie interesuje nazwa zmiennej, tylko jej **typ**. Co może być kłopotliwe - jeżeli konstruktor przyjmuje dwa parametry o typie `int`, `Dagger2` ich nie rozróżni. Jest to możliwe z kilkoma dodatkowymi adnotacjami. Elementy przedstawione do tej pory są wystarczające nac potrzeby tych zajęć, więc tutaj zakończymy omawianie biblioteki `Dagger2`.